In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, ConcatDataset, DataLoader
from PIL import Image
import os
import shutil

# Define transforms for data preprocessing (resize to 128x128)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 数据增强，随机水平翻转，随机旋转15度，随机亮度、对比度
random_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    # transforms.RandomErasing(p=0.5, scale=(0.02, 0.33)),
    # transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset class for food images
class FoodDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.image_paths = []
        self.labels = []
        
        for class_idx, class_name in enumerate(self.classes):
            class_path = os.path.join(root_dir, class_name)
            if os.path.isdir(class_path):
                for img_name in os.listdir(class_path):
                    if img_name.endswith(('.jpg', '.jpeg', '.png')):
                        self.image_paths.append(os.path.join(class_path, img_name))
                        self.labels.append(class_idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

# Create dataset and dataloader
train_dataset = FoodDataset(root_dir='food/training/labeled', transform=random_transform)
trylabeled_dataset = FoodDataset(root_dir='food/training/trylabeled', transform=transform)

# 使用ConcatDataset合并
combined_train_dataset = ConcatDataset([train_dataset, trylabeled_dataset])

train_loader = DataLoader(combined_train_dataset, batch_size=64, shuffle=True)

# Create validation dataset and dataloader
val_dataset = FoodDataset(root_dir='food/validation', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


# Define the CNN model
class FoodClassifier(nn.Module):
    def __init__(self, num_classes=11):
        super(FoodClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 16 * 16, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Initialize model, loss function, and optimizer
# For Apple Silicon (M1/M2/M3), use MPS device if available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

model = FoodClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

def train(model, train_loader, criterion, optimizer, epoch, num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0

def validate(model, val_loader, criterion):
    model.eval()
    accuracy = 0
    avg_loss = 0
    correct = 0
    running_loss = 0.0
    size = len(val_loader.dataset)
    num_batches = len(val_loader)
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            
            probabilities = F.softmax(outputs, dim=1)  # 计算每个类别的概率
            _, predicted = torch.max(probabilities, 1) 
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / size
    avg_loss = running_loss / num_batches
    
    print(f'Validation Accuracy: {accuracy:.2f}%')
    print(f'Validation Loss: {avg_loss:.4f}')
    
    return accuracy, avg_loss

Using device: mps


In [12]:
# Training loop
num_epochs = 50
min_accuracy = 0

for epoch in range(num_epochs):
    train(model, train_loader, criterion, optimizer, epoch, num_epochs)
    accuracy, loss = validate(model, val_loader, criterion)
    if accuracy > min_accuracy:
        min_accuracy = accuracy
        torch.save(model.state_dict(), 'food_classifier.pth')

print("Finished!")


Epoch [1/50], Step [10/57], Loss: 0.2464
Epoch [1/50], Step [20/57], Loss: 0.2381
Epoch [1/50], Step [30/57], Loss: 0.2367
Epoch [1/50], Step [40/57], Loss: 0.2310
Epoch [1/50], Step [50/57], Loss: 0.2323
Validation Accuracy: 13.79%
Validation Loss: 2.2777
Epoch [2/50], Step [10/57], Loss: 0.2311
Epoch [2/50], Step [20/57], Loss: 0.2309
Epoch [2/50], Step [30/57], Loss: 0.2253
Epoch [2/50], Step [40/57], Loss: 0.2240
Epoch [2/50], Step [50/57], Loss: 0.2242
Validation Accuracy: 15.15%
Validation Loss: 2.2547
Epoch [3/50], Step [10/57], Loss: 0.2258
Epoch [3/50], Step [20/57], Loss: 0.2186
Epoch [3/50], Step [30/57], Loss: 0.2233
Epoch [3/50], Step [40/57], Loss: 0.2179
Epoch [3/50], Step [50/57], Loss: 0.2219
Validation Accuracy: 13.48%
Validation Loss: 2.2734
Epoch [4/50], Step [10/57], Loss: 0.2129
Epoch [4/50], Step [20/57], Loss: 0.2177
Epoch [4/50], Step [30/57], Loss: 0.2205
Epoch [4/50], Step [40/57], Loss: 0.2124
Epoch [4/50], Step [50/57], Loss: 0.2093
Validation Accuracy: 16.

In [5]:
# Load the saved model
model.load_state_dict(torch.load('food_classifier.pth'))
model.eval()


# Create output directory if it doesn't exist
output_base_dir = 'food/training/trylabeled'

# 如果目录存在，则删除
if os.path.exists(output_base_dir):
    shutil.rmtree(output_base_dir)

if not os.path.exists(output_base_dir):
    os.makedirs(output_base_dir)

# Create class subdirectories
for i in range(11):  # 11 food classes
    class_dir = os.path.join(output_base_dir, str(i))
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

# Process images
confidence_threshold = 0.99  # Adjust this threshold as needed
input_dir = 'food/training/unlabeled/00'

for filename in os.listdir(input_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Load and preprocess image
        img_path = os.path.join(input_dir, filename)
        image = Image.open(img_path).convert('RGB')
        img_tensor = transform(image).unsqueeze(0).to(device)
        
        # Get prediction
        with torch.no_grad():
            outputs = model(img_tensor)
            probabilities = F.softmax(outputs, dim=1)
            max_prob, predicted_class = torch.max(probabilities, 1)
            
            # If confidence is high enough, copy to class directory
            if max_prob.item() > confidence_threshold:
                predicted_label = predicted_class.item()
                dest_path = os.path.join(output_base_dir, str(predicted_label), filename)
                shutil.copy2(img_path, dest_path)
                print(f'Copied {filename} to class {predicted_label} with confidence {max_prob.item():.4f}')

print("Classification complete!")



Copied 1353.jpg to class 4 with confidence 0.9997
Copied 5647.jpg to class 10 with confidence 0.9902
Copied 2128.jpg to class 9 with confidence 0.9994
Copied 6428.jpg to class 9 with confidence 0.9993
Copied 5121.jpg to class 10 with confidence 0.9996
Copied 2302.jpg to class 9 with confidence 1.0000
Copied 0501.jpg to class 9 with confidence 0.9999
Copied 5323.jpg to class 9 with confidence 1.0000
Copied 4798.jpg to class 9 with confidence 1.0000
Copied 1019.jpg to class 9 with confidence 0.9997
Copied 4820.jpg to class 9 with confidence 0.9996
Copied 5280.jpg to class 9 with confidence 0.9959
Copied 4405.jpg to class 9 with confidence 1.0000
Copied 2074.jpg to class 5 with confidence 0.9917
Copied 0663.jpg to class 6 with confidence 0.9906
Copied 0688.jpg to class 9 with confidence 1.0000
Copied 4410.jpg to class 9 with confidence 0.9991
Copied 0676.jpg to class 9 with confidence 0.9997
Copied 5256.jpg to class 4 with confidence 0.9961
Copied 0448.jpg to class 1 with confidence 0.992